In [ ]:
from datasets import load_from_disk

# Dataset'i diskten yüklüyoruz
dataset = load_from_disk("my_dataset")


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_wnut = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
example = dataset["test"][0]

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

import numpy as np

label_list = [
    "B-PER", "I-PER", "B-OPE", "I-OPE", "B-BANK", "I-BANK", "B-APP", "I-APP",
    "B-LOC", "I-LOC", "B-NET", "I-NET", "B-PAY", "I-PAY", "B-PKG", "I-PKG",
    "B-LIN", "I-LIN", "B-SER", "I-SER", "B-OTH", "I-OTH", "B-DATE", "I-DATE",
    "B-NUM", "I-NUM", "O"
]

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# Create id2label and label2id mappings
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(label_list), id2label=id2label, label2id=label2id
)

In [ ]:
# prompt: gpu ram i temizle

import torch
import gc

# Clear CUDA cache
torch.cuda.empty_cache()

# Collect garbage
gc.collect()

# You can also delete variables that are no longer needed explicitly
del dataset
del tokenizer
del tokenized_wnut
del data_collator
del example
del seqeval
del labels
del model

# Clear CUDA cache again, just in case
torch.cuda.empty_cache()


In [ ]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:

# Evaluate the model on the test set
test_results = trainer.evaluate(tokenized_wnut["test"])

# Print the overall metrics
print("Overall Test Results:")
print(f"  Precision: {test_results['eval_precision']:.4f}")
print(f"  Recall: {test_results['eval_recall']:.4f}")
print(f"  F1: {test_results['eval_f1']:.4f}")
print(f"  Accuracy: {test_results['eval_accuracy']:.4f}")

# Compute metrics per class
predictions, labels, _ = trainer.predict(tokenized_wnut["test"])
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = seqeval.compute(predictions=true_predictions, references=true_labels)

# Print per-class metrics
print("\nPer-Class Test Results:")
for label, metrics in results.items():
    if label not in ["overall_precision", "overall_recall", "overall_f1", "overall_accuracy"]:
      print(f"  {label}:")
      print(f"    Precision: {metrics['precision']:.4f}")
      print(f"    Recall: {metrics['recall']:.4f}")
      print(f"    F1: {metrics['f1']:.4f}")
      print(f"    Number: {metrics['number']}")


In [ ]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [ ]:
from transformers import pipeline

classifier = pipeline("ner", model="stevhliu/my_awesome_wnut_model")
classifier(text)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_wnut_model")
inputs = tokenizer(text, return_tensors="pt")
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("stevhliu/my_awesome_wnut_model")
with torch.no_grad():
    logits = model(**inputs).logits
predictions = torch.argmax(logits, dim=2)

predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

predicted_token_class